In [ ]:
## First time set up
# !pip install ultralytics
# !pip install --upgrade pip
# !pip install jupyterlab ultralytics

  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.3

In [32]:
import os
import random
import shutil


In [ ]:
# Change these paths and crop parameters before running
input_dir = "/path/to/input"      # folder containing .mp4 files
output_dir = "/path/to/output"    # folder to save cropped frames
crop_params = "640:480:100:50"    # format: width:height:x_offset:y_offset

!mkdir -p "{output_dir}"

# Loop through .mp4 files, extract frame at 1s, crop, save to output dir
!for file in "{input_dir}"/*.mp4; do \
    filename=$(basename "$file" .mp4); \
    ffmpeg -ss 00:00:01 -i "$file" -vframes 1 -vf "crop={crop_params}" "{output_dir}/${filename}_cropped_frame.jpg"; \
done

In [40]:
import os
import shutil
import random

# --- Paths ---
images_dir = '../data/cvat_out/images'    # Your images folder
labels_dir = '../data/cvat_out/labels'    # Your labels folder
output_dir = '../data/cvat_out/split_dataset'   # New folder for split dataset

# --- Split proportions (by number of labels) ---
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15  # will be the remainder

# Create output folders
for split in ['train', 'val', 'test']:
    os.makedirs(f'{output_dir}/images/{split}', exist_ok=True)
    os.makedirs(f'{output_dir}/labels/{split}', exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

# Count labels for each image
image_label_counts = []
total_labels = 0
for img in image_files:
    label_file = img.rsplit('.', 1)[0] + '.txt'
    label_path = os.path.join(labels_dir, label_file)
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            count = len(f.readlines())
    else:
        count = 0
    image_label_counts.append((img, count))
    total_labels += count

# Shuffle images for randomness
random.shuffle(image_label_counts)

# Target counts for each split
target_train = int(total_labels * train_ratio)
target_val = int(total_labels * val_ratio)
target_test = total_labels - target_train - target_val

splits = {'train': [], 'val': [], 'test': []}
label_counts = {'train': 0, 'val': 0, 'test': 0}

# Assign images to splits based on label counts
for img, count in image_label_counts:
    if label_counts['train'] + count <= target_train:
        splits['train'].append(img)
        label_counts['train'] += count
    elif label_counts['val'] + count <= target_val:
        splits['val'].append(img)
        label_counts['val'] += count
    else:
        splits['test'].append(img)
        label_counts['test'] += count

# Copy files
for split, files in splits.items():
    for img_file in files:
        src_img = os.path.join(images_dir, img_file)
        dst_img = os.path.join(output_dir, 'images', split, img_file)
        shutil.copy(src_img, dst_img)

        label_file = img_file.rsplit('.', 1)[0] + '.txt'
        src_lbl = os.path.join(labels_dir, label_file)
        dst_lbl = os.path.join(output_dir, 'labels', split, label_file)
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, dst_lbl)

print(f"Data split complete!")
print(f"Train: {len(splits['train'])} images, {label_counts['train']} labels")
print(f"Val:   {len(splits['val'])} images, {label_counts['val']} labels")
print(f"Test:  {len(splits['test'])} images, {label_counts['test']} labels")

Data split complete!
Train: 41 images, 1014 labels
Val:   7 images, 214 labels
Test:  9 images, 223 labels


In [41]:
#Get counts of annotations from a folder of labels
import glob

label_files = glob.glob('../data/cvat_out/split_dataset/labels/train/*.txt')
label_count = 0
for file in label_files:
    with open(file) as f:
        lines = f.readlines()
        label_count += len(lines)

print(f'Total annotations in training set: {label_count}')

Total annotations in training set: 1014


In [42]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')  # Use 'n'ano model for speed, or 's' for small

model.train(
    data='../data/cvat_out/data.yaml',
    epochs=50,
    imgsz=1280,
    batch=16,
    patience=10, # early stopping if no improvement after 10 epochs
    project='../models',
    name='yolov8s_surf_aug_13'
)

New https://pypi.org/project/ultralytics/8.3.179 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.169 🚀 Python-3.11.7 torch-2.7.1 CPU (Apple M3 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/cvat_out/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_surf_aug_13, nbs=64, nms=False, opset=None, optimize=False, o

train: Scanning /Users/jfs-m3/Documents/DS/surf_count/data/cvat_out/split_dataset/labels/train... 41 images, 0 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:00<00:00, 2512.73it/s]

train: New cache created: /Users/jfs-m3/Documents/DS/surf_count/data/cvat_out/split_dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 284.3±127.0 MB/s, size: 30.1 KB)



val: Scanning /Users/jfs-m3/Documents/DS/surf_count/data/cvat_out/split_dataset/labels/val... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<00:00, 2978.91it/s]

val: New cache created: /Users/jfs-m3/Documents/DS/surf_count/data/cvat_out/split_dataset/labels/val.cache
Plotting labels to ../models/yolov8s_surf_aug_13/labels.jpg... 


...

                   all          7        214      0.488      0.528      0.428       0.11
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

19 epochs completed in 0.496 hours.


Optimizer stripped from ../models/yolov8s_surf_aug_13/weights/last.pt, 22.5MB
Optimizer stripped from ../models/yolov8s_surf_aug_13/weights/best.pt, 22.5MB

Validating ../models/yolov8s_surf_aug_13/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.7 torch-2.7.1 CPU (Apple M3 Pro)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.01it/s]


                   all          7        214      0.757      0.668      0.698      0.239
Speed: 0.3ms preprocess, 65.8ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to ../models/yolov8s_surf_aug_13


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x31848b090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [43]:
model.val()

Ultralytics 8.3.169 🚀 Python-3.11.7 torch-2.7.1 CPU (Apple M3 Pro)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 152.1±27.2 MB/s, size: 29.5 KB)


val: Scanning /Users/jfs-m3/Documents/DS/surf_count/data/cvat_out/split_dataset/labels/val.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


                   all          7        214      0.757      0.668      0.698      0.239
Speed: 0.7ms preprocess, 71.7ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to ../models/yolov8s_surf_aug_132


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x1289bec90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [ ]:
# Check for missing labels or images
# import os

# image_files = set(f.replace('.jpg', '') for f in os.listdir('../data/cvat_out/split_dataset/images/train'))
# label_files = set(f.replace('.txt', '') for f in os.listdir('../data/cvat_out/split_dataset/labels/train'))

# print("Missing labels:", image_files - label_files)
# print("Missing images:", label_files - image_files)

Missing labels: {'20250719_0845_frame_01', '20250719_1144_frame_05', 'fr_05_2', '20250719_1244_frame_02', 'fr_03_5', '20250719_1144_frame_03'}
Missing images: set()


In [31]:
# for running additional images on same model
results = model.predict(source='../data/cvat_out/split_dataset/images/test', save=True, conf=0.4)


image 1/6 /Users/jfs-m3/Documents/DS/surf_count/code/../data/cvat_out/split_dataset/images/test/20250719_1044.jpg: 192x1280 40 Surfers, 61.9ms
image 2/6 /Users/jfs-m3/Documents/DS/surf_count/code/../data/cvat_out/split_dataset/images/test/20250719_1645.jpg: 192x1280 2 Surfers, 49.9ms
image 3/6 /Users/jfs-m3/Documents/DS/surf_count/code/../data/cvat_out/split_dataset/images/test/jacks_20250809_0605.jpg: 192x1280 12 Surfers, 50.3ms
image 4/6 /Users/jfs-m3/Documents/DS/surf_count/code/../data/cvat_out/split_dataset/images/test/jacks_20250809_1358.jpg: 192x1280 18 Surfers, 49.8ms
image 5/6 /Users/jfs-m3/Documents/DS/surf_count/code/../data/cvat_out/split_dataset/images/test/jacks_20250809_1501.jpg: 192x1280 27 Surfers, 49.3ms
image 6/6 /Users/jfs-m3/Documents/DS/surf_count/code/../data/cvat_out/split_dataset/images/test/jacks_20250809_1900.jpg: 192x1280 22 Surfers, 49.1ms
Speed: 0.6ms preprocess, 51.7ms inference, 0.3ms postprocess per image at shape (1, 3, 192, 1280)
Results saved to ../